In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import time
import sklearn
import warnings
import pickle

warnings.filterwarnings(action='ignore', category=FutureWarning)

pd.__version__, sklearn.__version__

('2.1.1', '1.2.1')

In [2]:
SCRATCH_DIR = "/scratch/siads696f23_class_root/siads696f23_class/psollars"

In [3]:
start_time = time.time()
all_features_2019 = pd.read_parquet(f"{SCRATCH_DIR}/all_features_2019.parquet")
end_time = time.time()

print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

Elapsed time: 19.3678 seconds


In [3]:
# Feature selection from "feature_selection_2019_UA-for-real.ipynb"

num_cols = [
    "Quarter",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "CRSDepTime",
    "CRSArrTime",
    "CRSElapsedTime",
    "Distance",
    "DistanceGroup",
    "YEAR MFR",
    "HORSEPOWER",
    "THRUST",
    "NO-SEATS",
    "Origin_LATITUDE",
    "Origin_LONGITUDE",
    "Dest_LATITUDE",
    "Dest_LONGITUDE",
]

cat_cols = [
#     "FlightDate",
    "Reporting_Airline", # we need this to filter out the dataset later
    "Tail_Number",
    "Origin",
    "DestCityMarketID",
    "Dest",
    "N-NUMBER",
    "MFR MDL CODE",
    "ENG MFR MDL",
    "TYPE REGISTRANT",
    "NAME",
    "CERTIFICATION",
    "TYPE ENGINE",
    "STATUS CODE",
    "MODE S CODE",
    "MODE S CODE HEX",
    "CODE",
    "MFR",
    "MODEL",
    "TYPE",
    "CODE_aircraft",
    "MFR_aircraft",
    "MODEL_aircraft",
    "TYPE-ENG",
    "BUILD-CERT-IND",
    "faa_registration_origin",
    "Origin_DISPLAY_AIRPORT_NAME",
    "Origin_DISPLAY_AIRPORT_CITY_NAME_FULL",
    "Dest_DISPLAY_AIRPORT_NAME",
    "Dest_DISPLAY_AIRPORT_CITY_NAME_FULL",
]

In [59]:
predictive_features = all_features_2019[all_features_2019["Cancelled"].eq(0) & all_features_2019["Diverted"].eq(0)].copy()

predictive_features["delayed"] = (predictive_features["DepDel15"].eq(1)) | (predictive_features["ArrDel15"].eq(1))

predictive_features = predictive_features[num_cols + cat_cols + ["delayed"]]

# Column YEAR MFR contains non-convertible values.
predictive_features["YEAR MFR"] = predictive_features["YEAR MFR"].replace(r'^\s*$', np.nan, regex=True)

predictive_features = predictive_features.dropna(axis="index", how=("any"))

predictive_features.to_parquet(f"{SCRATCH_DIR}/11_predictive_features_2019.parquet")

In [4]:
predictive_features = pd.read_parquet(f"{SCRATCH_DIR}/11_predictive_features_2019.parquet")

predictive_features

,Quarter,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,CRSElapsedTime,Distance,DistanceGroup,YEAR MFR,...,MFR_aircraft,MODEL_aircraft,TYPE-ENG,BUILD-CERT-IND,faa_registration_origin,Origin_DISPLAY_AIRPORT_NAME,Origin_DISPLAY_AIRPORT_CITY_NAME_FULL,Dest_DISPLAY_AIRPORT_NAME,Dest_DISPLAY_AIRPORT_CITY_NAME_FULL,delayed
24733033,1,1,4,5,1400,1510,70,190,1,2003,...,BOMBARDIER INC,CL-600-2B19,5.0,0.0,2023.0,San Luis County Regional,"San Luis Obispo, CA",San Francisco International,"San Francisco, CA",False
24733034,1,1,4,5,0935,1118,103,438,2,2002,...,BOMBARDIER INC,CL-600-2B19,5.0,0.0,2023.0,George Bush Intercontinental/Houston,"Houston, TX",Northwest Arkansas National,"Fayetteville, AR",False
24733035,1,1,4,5,0643,0855,132,513,3,2002,...,BOMBARDIER INC,CL-600-2B19,5.0,0.0,2023.0,Springfield-Branson National,"Springfield, MO",George Bush Intercontinental/Houston,"Houston, TX",False
24733036,1,1,4,5,1335,1433,118,576,3,2002,...,BOMBARDIER INC,CL-600-2B19,5.0,0.0,2023.0,Sloulin Field International,"Williston, ND",Denver International,"Denver, CO",False
24733037,1,1,4,5,0836,1040,184,896,4,2014,...,EMBRAER S A,ERJ 170-200 LR,5.0,0.0,2023.0,General Mitchell International,"Milwaukee, WI",Denver International,"Denver, CO",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32288695,4,12,1,7,0600,0802,122,762,4,2018,...,AIRBUS,A321-211,5.0,0.0,2023.0,Hartsfield-Jackson Atlanta International,"Atlanta, GA",LaGuardia,"New York, NY",False
32288696,4,12,1,7,1209,1438,209,1310,6,1999,...,BOEING,757-232,5.0,0.0,2023.0,Orlando International,"Orlando, FL",Minneapolis-St Paul International,"Minneapolis, MN",True
32288697,4,12,1,7,0650,1110,200,1310,6,1999,...,BOEING,757-232,5.0,0.0,2023.0,Minneapolis-St Paul International,"Minneapolis, MN",Orlando International,"Orlando, FL",True
32288698,4,12,1,7,1639,1932,293,1851,8,2000,...,BOEING,757-232,5.0,0.0,2023.0,Ronald Reagan Washington National,"Washington, DC",Salt Lake City International,"Salt Lake City, UT",False


In [9]:
# start_time = time.time()

# delta_df = all_features_2019[all_features_2019["Reporting_Airline"].eq("DL")].drop("Reporting_Airline", axis=1)

# df = delta_df[delta_df["Cancelled"].eq(0) & delta_df["Diverted"].eq(0)].copy()

# # df = df[df["Origin"].eq("ORD") | df["Dest"].eq("ORD")]

# df["delayed"] = (df["DepDel15"].eq(1)) | (df["ArrDel15"].eq(1))

# # Column YEAR MFR contains non-convertible values.
# df["YEAR MFR"] = df["YEAR MFR"].replace(r'^\s*$', np.nan, regex=True)

# df_features = df[num_cols + cat_cols + ["delayed"]]

# df_features = df_features.dropna(axis="index", how=("any"))

# end_time = time.time()

# print(f"Removed {len(delta_df) - len(df)}  rows")

# print(f"Removed {len(df) - len(df_features)} NaN rows")

# print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

# # df_features

Removed 4017  rows
Removed 128114 NaN rows
Elapsed time: 5.8959 seconds


In [6]:
predictive_features.value_counts("delayed")

# United delay labels
# False    114714
# True      39646

# Delta delay labels
# False    715015
# True     159058

delayed
False    5410064
True     1601796
Name: count, dtype: int64

In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


def preprocess_data(preprocess_df):
    start_time = time.time()
    
    X = preprocess_df.drop("delayed", axis=1)
    y = preprocess_df["delayed"]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(), cat_cols),
        ]
    )

    X_transformed = preprocessor.fit_transform(X)
    
    num_feature_names = np.array(num_cols)
    cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=cat_cols)

    feature_names = np.concatenate([num_feature_names, cat_feature_names])

#     feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=cat_cols)
    
    end_time = time.time()
    
    print(f"Finished Preprocessing: {(end_time - start_time):.4f} seconds")
    
    return X_transformed, y, feature_names

In [6]:
predictive_features_2019_X, predictive_features_2019_y, predictive_features_2019_feature_names = preprocess_data(predictive_features)

Finished Preprocessing: 91.4650 seconds


In [7]:
predictive_features_2019_feature_names

array(['Quarter', 'Month', 'DayofMonth', ...,
       'Dest_DISPLAY_AIRPORT_CITY_NAME_FULL_Wrangell, AK',
       'Dest_DISPLAY_AIRPORT_CITY_NAME_FULL_Yakutat, AK',
       'Dest_DISPLAY_AIRPORT_CITY_NAME_FULL_Yuma, AZ'], dtype=object)

In [8]:
with open(f"{SCRATCH_DIR}/11_predictive_features_2019_X.pkl", "wb") as f:
    pickle.dump(predictive_features_2019_X, f)
    
with open(f"{SCRATCH_DIR}/11_predictive_features_2019_y.pkl", "wb") as f:
    pickle.dump(predictive_features_2019_y, f)
    
with open(f"{SCRATCH_DIR}/11_predictive_features_2019_feature_names.pkl", "wb") as f:
    pickle.dump(predictive_features_2019_feature_names, f)

In [11]:
with open(f"{SCRATCH_DIR}/11_predictive_features_2019_X.pkl", "rb") as f:
    predictive_features_2019_X = pickle.load(f)
    
with open(f"{SCRATCH_DIR}/11_predictive_features_2019_y.pkl", "rb") as f:
    predictive_features_2019_y = pickle.load(f)
    
with open(f"{SCRATCH_DIR}/11_predictive_features_2019_feature_names.pkl", "rb") as f:
    predictive_features_2019_feature_names = pickle.load(f)

In [9]:
np.where(predictive_features_2019_feature_names == "Reporting_Airline_DL")[0][0]

21

In [10]:
predictive_features_2019_X[:, 21].toarray().ravel()

array([0., 0., 0., ..., 1., 1., 1.])

In [11]:
def filter_x_y_by_airline(filter_X, filter_y, airline):
    index = np.where(predictive_features_2019_feature_names == f"Reporting_Airline_{airline}")[0][0]
    
    rows = filter_X[:, index].toarray().ravel() == 1
    
    X_airline = filter_X[rows]
    y_airline = filter_y[rows]
    
    return X_airline, y_airline
    
#     df = df[df["Origin"].eq("ORD") | df["Dest"].eq("ORD")]

In [12]:
X, y = filter_x_y_by_airline(predictive_features_2019_X, predictive_features_2019_y, "DL")


In [14]:
import xgboost as xgb


def fit_xg_boost(fit_X_train, fit_y_train):
    start_time = time.time()
    
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'scale_pos_weight': sum(y_train == 0) / sum(y_train == 1),  # Handle imbalance
    }


    bst = xgb.train(params, dtrain)
    
    end_time = time.time()
    
    print(f"Finished Fitting Model: {(end_time - start_time):.4f} seconds")
    
    return bst

In [13]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


def fit_grid_search_xg_boost(fit_X_train, fit_y_train):
    start_time = time.time()
    
    param_grid = {
        'learning_rate': [0.1, 0.2, 0.3],#0.01, 0.1, 0.2],
        'max_depth': [5, 7, 10],
        'n_estimators': [150, 250, 500],
        'subsample': [0.6, 0.7, 0.8],
        'colsample_bytree': [0.6, 0.7, 0.8],
        'scale_pos_weight': [sum(fit_y_train == 0) / sum(fit_y_train == 1)]  # Handle imbalance
    }

    xgb_clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
    
    grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid,
                           scoring='average_precision',  # AUC-PR
                           cv=5, n_jobs=-1)

    grid_search.fit(fit_X_train, y_train)
    
    end_time = time.time()
    
    print(f"Finished Fitting Model: {(end_time - start_time):.4f} seconds")
    
    return grid_search

In [ ]:
from sklearn.model_selection import train_test_split

start_time = time.time()

X, y = filter_x_y_by_airline(predictive_features_2019_X, predictive_features_2019_y, "UA")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

model = fit_grid_search_xg_boost(X_train, y_train)

end_time = time.time()

print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/psollars/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is 

In [ ]:
model.best_params_

In [ ]:
with open(f"{SCRATCH_DIR}/12_grid_search_xg_boost_2019_UA_ALL.pkl", "wb") as f:
    pickle.dump(model, f)

# Model Evaluation

In [16]:
with open(f"{SCRATCH_DIR}/12_grid_search_xg_boost_2019_UA_ALL.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
best_estimator = model.best_estimator_

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    precision_recall_curve,
)

# y_pred_raw = model.predict(dtest)

# # Make the predictions binary
# y_pred = [1 if y > 0.5 else 0 for y in y_pred]

y_pred = best_estimator.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(
    confusion_matrix=cm, display_labels=["Not Delayed", "Delayed"]
).plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
cm

# DL Confusion Matrix
# array([[97923, 45244],
#        [11049, 20599]])

In [ ]:
y_scores = model.best_estimator_.predict_proba(X_test)[:, 1]

precision, recall, _ = precision_recall_curve(y_test, y_scores)

plt.figure()
plt.step(recall, precision, where="post")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score


y_pred_proba = model.best_estimator_.predict_proba(X_test)

ra_score = roc_auc_score(y_test, y_pred, average="macro", multi_class="ovr")

ra_score

In [ ]:
fpr, tpr, _ = roc_curve(y_test, model.best_estimator_.predict_proba(X_test)[:, 1])

plt.figure()
plt.plot(fpr, tpr, color="darkorange", label="ROC curve")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

metrics_scores = {
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
}
df_metrics = pd.DataFrame(list(metrics_scores.items()), columns=["Metric", "Score"])

fig = px.bar(
    df_metrics,
    x="Score",
    y="Metric",
    orientation="h",
    color="Metric",
    text="Score",
    title="Evaluation Metrics",
)

fig.update_layout(showlegend=False)

fig.show()